# SPR 2026 - Qwen3 1.7B (Zero-Shot Classification)

## ✅ Compatível com competições que exigem Internet OFF

**Este notebook usa Qwen3-1.7B do Kaggle Models.**

### Configuração:

1. **NÃO precisa de internet** - funciona offline
2. Adicione o modelo: `QwenLM/Qwen3` → Variação `1.7B`
3. Settings → Accelerator → **GPU T4 x2** ou **P100**
4. Run All

### Estratégia:
- Zero-shot classification com prompt em português
- Classes BI-RADS: 0, 1, 2, 3, 4, 5, 6

---

In [ ]:
# =============================================================================
# SPR 2026 - QWEN3 1.7B ZERO-SHOT (OFFLINE)
# =============================================================================
# REQUER: Modelo "QwenLM/Qwen3" (variação 1.7B) adicionado ao notebook
# =============================================================================

import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - Qwen3 1.7B Zero-Shot (OFFLINE)")
print("="*60)

# ==== CONFIGURAÇÕES ====
SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

# Paths para modelos Kaggle (verificar qual está disponível)
MODEL_PATHS = [
    '/kaggle/input/qwen3/transformers/1.7b/1',
    '/kaggle/input/qwen3/transformers/1.7b',
    '/kaggle/input/qwen-3/transformers/1.7b/1',
]

MODEL_PATH = None
for path in MODEL_PATHS:
    if os.path.exists(path):
        MODEL_PATH = path
        break

if MODEL_PATH is None:
    # Listar o que está disponível
    print("\n⚠️ Modelo não encontrado. Datasets disponíveis:")
    for item in os.listdir('/kaggle/input'):
        print(f"  - {item}")
        try:
            for sub in os.listdir(f'/kaggle/input/{item}')[:3]:
                print(f"      └── {sub}")
        except:
            pass
    raise FileNotFoundError("Adicione o modelo QwenLM/Qwen3 (1.7B) ao notebook!")

USE_GPU = torch.cuda.is_available()
device = 'cuda' if USE_GPU else 'cpu'

print(f"✓ Modelo: {MODEL_PATH}")
print(f"✓ GPU disponível: {USE_GPU}")
print(f"✓ Device: {device}")

In [ ]:
# ==== CARREGAR DADOS ====
print("\n[1/4] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"    Train: {train.shape} | Test: {test.shape}")

# Ver distribuição das classes
print("\nDistribuição das classes no treino:")
print(train['target'].value_counts().sort_index())

In [ ]:
# ==== CARREGAR MODELO ====
print("\n[2/4] Carregando Qwen3 1.7B...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16 if USE_GPU else torch.float32,
    device_map='auto' if USE_GPU else None,
    trust_remote_code=True,
)

if not USE_GPU:
    model = model.to(device)

model.eval()
print("    ✓ Modelo carregado!")

In [ ]:
# ==== PROMPT TEMPLATE ====
PROMPT_TEMPLATE = """Você é um radiologista especialista em mamografia. 
Classifique o seguinte laudo de mamografia na categoria BI-RADS correta.

Categorias BI-RADS:
0 - Inconclusivo, necessita exames adicionais
1 - Negativo, sem achados
2 - Achados benignos
3 - Provavelmente benigno
4 - Suspeito de malignidade
5 - Altamente suspeito de malignidade
6 - Malignidade confirmada por biópsia

Laudo:
{report}

Responda APENAS com o número da categoria BI-RADS (0, 1, 2, 3, 4, 5 ou 6):"""

def classify_report(report, max_length=1024):
    """Classifica um laudo usando o LLM."""
    prompt = PROMPT_TEMPLATE.format(report=report[:2000])  # Truncar laudos muito longos
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    # Extrair número da resposta
    for char in response.strip():
        if char.isdigit() and int(char) in range(7):
            return int(char)
    
    # Fallback: classe mais comum
    return 2  # BI-RADS 2 é comum

# Testar com uma amostra
print("\nTestando classificação...")
sample_report = train.iloc[0]['report']
sample_pred = classify_report(sample_report)
print(f"    Predição: {sample_pred} | Real: {train.iloc[0]['target']}")

In [ ]:
# ==== CLASSIFICAR TESTE ====
print("\n[3/4] Classificando dados de teste...")

predictions = []
for idx, row in tqdm(test.iterrows(), total=len(test), desc="Classificando"):
    pred = classify_report(row['report'])
    predictions.append(pred)
    
    # Liberar memória GPU periodicamente
    if idx % 100 == 0 and USE_GPU:
        torch.cuda.empty_cache()

print("    ✓ Classificação concluída!")

In [ ]:
# ==== SUBMISSÃO ====
print("\n[4/4] Gerando submissão...")

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})
submission.to_csv('submission.csv', index=False)

print("="*60)
print("✅ CONCLUÍDO - submission.csv criado!")
print("="*60)
print("\nDistribuição das predições:")
print(submission['target'].value_counts().sort_index())